In [ ]:
# Make sure pip is up to date
!pip install --upgrade pip

# Install OpenCV
!pip3 install opencv-python-headless numpy

# Install PyTorch (CPU-only for Pi 4B)
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Install ultralytics (YOLO v8+)
!pip3 install ultralytics


In [ ]:
# Uncomment if you haven't installed yet
# !pip install --upgrade pip
# !pip install opencv-python-headless numpy ultralytics torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import cv2
import numpy as np
from ultralytics import YOLO

print("Dependencies imported successfully")


In [ ]:
# Path to your trained model
MODEL_PATH = "trained_yolo_model/ODM-ver5.pt"

# Load YOLO model
model = YOLO(MODEL_PATH)
print("YOLO model loaded successfully")


In [ ]:
# Calibration parameters
KNOWN_WIDTH_INCH = 1.0      # block width
KNOWN_DISTANCE_INCH = 12.0  # place block at this distance for calibration

cap = cv2.VideoCapture(0)
FOCAL_LENGTH = None

print("Calibration: place a single block exactly 12 inches away and press SPACE")

while True:
    ret, frame = cap.read()
    if not ret:
        continue
    
    results = model(frame)
    annotated = results[0].plot()
    
    cv2.imshow("Calibration", annotated)
    key = cv2.waitKey(1)
    
    if key & 0xFF == 32:  # SPACE key
        boxes = results[0].boxes.xywh  # x_center, y_center, w, h
        if len(boxes) > 0:
            w_pixels = boxes[0][2].item()
            FOCAL_LENGTH = (w_pixels * KNOWN_DISTANCE_INCH) / KNOWN_WIDTH_INCH
            print(f"✅ Focal length calibrated: {FOCAL_LENGTH:.2f} pixels")
            break
        else:
            print("⚠️ No block detected. Try again")
    elif key & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
if FOCAL_LENGTH is None:
    print("⚠️ Focal length not calibrated. Run calibration first.")
else:
    cap = cv2.VideoCapture(0)
    print("Starting live distance estimation. Press ESC to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        results = model(frame)
        annotated = results[0].plot()

        for box in results[0].boxes.xywh:
            w_pixels = box[2].item()
            distance_inch = (KNOWN_WIDTH_INCH * FOCAL_LENGTH) / w_pixels
            x, y = int(box[0]), int(box[1])
            cv2.putText(annotated, f"{distance_inch:.2f} in", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("YOLO Distance Estimation", annotated)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
            break

    cap.release()
    cv2.destroyAllWindows()
